In [8]:
#Getting dataframe setup
import pandas as pd

#Causal inference library
from dowhy import CausalModel
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#Getting libraries for Logistic Regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

In [15]:
#Load dataframe
df = pd.read_csv('./out/df.csv')

In [16]:
#Add kill participation 
df['adc_killpart'] = ((df['adc_kills'] + df['adc_assists']) / df['kills']) 
df['jng_killpart'] = ((df['jng_kills'] + df['jng_assists']) / df['kills']) 

#add pre 10 kill participation for jng
df['jng_killpart_at10'] = ((df['jng_killsat10']+df['jng_assistsat10']) / df['killsat10'])

In [24]:
#Define performance features for both roles 
#mb add adc_totaldamage
#for now add only killpart and maybe show trough other stuff that kills also should be add idk
adc_performance_features = ['adc_kills', 'adc_assists', 'adc_dpm', 'adc_damagetochampions', 'adc_damageshare', 'adc_total_cs', 'adc_damagetakenperminute']
jng_performance_features = ['jng_killsat10', 'jng_assistsat10', 'jng_total_cs', 'jng_dpm', 'jng_deaths', 'jng_kills'] 

In [25]:
#Define the causal graph
#TODO: Draw the proper graph up again please so I can be sure of results
causal_graph = '''
digraph {
    wardsplaced -> visionscore;
    wardskilled -> visionscore;
    adc_damageshare -> kills;
    adc_damagetakenperminute -> kills;
    adc_dpm -> kills;
    jng_dpm -> kills;
    wcpm -> visionscore;
    wppm -> visionscore;
    visionscore -> gamelength;
    win_prob -> gamelength;
    rating_before -> win_prob;
    opp_rating_before -> win_prob;
    win_prob -> golddiffat15;
    golddiffat15 -> result;
    result -> rating_after;
    rating_before -> rating_after;
    opp_rating_before -> rating_after;
    win_prob -> result;
    rating_after -> rating_gain;
    visionscore -> kills; 
    visionscore -> deaths; 
    visionscore -> assists;
    kills -> golddiffat15;
    adc_kills -> kills;
    jng_kills -> kills;
    side_adv -> win_prob;
    damagetochampions -> kills;
    adc_damagetochampions -> kills;
    jng_damagetochampions -> kills;  
}
'''

In [26]:
def causal_inference(treatment, outcome, method_name = 'backdoor.linear_regression', data = df):
    #Create causal model
    model = CausalModel(
        data = data,
        treatment = treatment,
        outcome = outcome,
        graph = causal_graph
    )

    #Identify causal effects
    identified_estimand = model.identify_effect()

    #Estimate the causal effect using backdoor adjustment with linearregression
    estimate = model.estimate_effect(
        identified_estimand,
        method_name = method_name
    )
    
    refute_results = None
    #refute_results = model.refute_estimate(identified_estimand, estimate, method_name='random_common_cause')

    return estimate, refute_results

In [27]:
print(df.isnull().sum())

gameid                   0
teamid                   0
result                   0
side                     0
kills                    0
                     ...  
rating_after             0
rating_gain              0
adc_killpart           481
jng_killpart           481
jng_killpart_at10    33509
Length: 64, dtype: int64


In [28]:
#TODO Note the not fully complete graph since im not sure about killpart etc so its not in the graph right now 
for feature in adc_performance_features:
    estimate, _ = causal_inference(feature, 'result')
    print(f'Estimate for the causal effect of {feature} on the result: {estimate.value}')

for feature in jng_performance_features:
    estimate, _ = causal_inference(feature, 'result')
    print(f'Estimate for the causal effect of {feature} on the result: {estimate.value}')


Estimate for the causal effect of adc_kills on the result: 0.08581466544537478


NetworkXError: The node adc_assists is not in the digraph.